# Concordance Analysis: Positive SPECT/CT and 4D CT Scans

### Objective

Evaluate **localization concordance with surgical findings** for patients with **positive SPECT/CT scans** and **positive 4D CT scans**, assessing **each modality’s diagnostic accuracy individually**.

> **Note:** Analyses were performed **separately for each modality**, not as a direct comparison between them.

### Data Source

- `side_concordance_results.xlsx` containing pre-operative imaging results and surgical outcomes.

### Outputs

- Concordance rates (% exact gland or side match) calculated **individually** for:
  - SPECT/CT scans
  - 4D CT scans
- Interpretation notes for manuscript reporting
 ---

In [2]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_cleaned_with_hyperplasia_concordance.xlsx')
import matplotlib.pyplot as plt


In [10]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'

In [11]:
def check_concordance(row, modality_col):
    imaging = row[modality_col]
    surgical = row['Surgical Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    # Exact gland match
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    # Side matches
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'

In [ ]:
# Define modality
modality = '4D CT Scan'
imaging_side_col = '4D CT Imaging Side'

# Filter to scans that localized (Left or Right)
localized_4dct = preop_imaging_df[preop_imaging_df[imaging_side_col].isin(['Left', 'Right'])].copy()
total_localized_4dct = localized_4dct.shape[0]

# Apply concordance classification with correct arguments
localized_4dct['Calculated Concordance'] = localized_4dct.apply(
    lambda row: check_concordance(row, modality), axis=1
)

# Calculate total concordant (either exact or side match)
total_concordant = localized_4dct['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()

# Calculate percentage
side_concordance_percentage_4dct = (total_concordant / total_localized_4dct * 100) if total_localized_4dct > 0 else None

# Create results dataframe
side_concordance_4dct_df = pd.DataFrame({
    'Metric': [
        'Total 4D CT scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_4dct,
        f"{total_concordant} ({side_concordance_percentage_4dct:.2f}%)"
    ]
})

# Display results
display(side_concordance_4dct_df)

,Metric,Value
0,Total 4D CT scans with localization,171
1,Scans matching surgical side or exact gland,146 (85.38%)


In [17]:
# Define modality
modality = 'SPECT/CT'
imaging_side_col = 'SPECT Imaging Side'

# Extract imaging side
preop_imaging_df[imaging_side_col] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized (Left or Right)
localized_spect = preop_imaging_df[preop_imaging_df[imaging_side_col].isin(['Left', 'Right'])].copy()
total_localized_spect = localized_spect.shape[0]


# Apply generalized check_concordance function with modality argument
localized_spect['Calculated Concordance'] = localized_spect.apply(
    lambda row: check_concordance(row, modality), axis=1
)

# Count total concordant (either exact or side match)
total_concordant = localized_spect['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()

# Calculate percentage
side_concordance_percentage_spect = (total_concordant / total_localized_spect * 100) if total_localized_spect > 0 else None

# Create results dataframe
side_concordance_spect_df = pd.DataFrame({
    'Metric': [
        'Total SPECT/CT scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_spect,
        f"{total_concordant} ({side_concordance_percentage_spect:.2f}%)"
    ]
})

# Display results
display(side_concordance_spect_df)

,Metric,Value
0,Total SPECT/CT scans with localization,177
1,Scans matching surgical side or exact gland,163 (92.09%)


In [14]:
# Combine SPECT/CT and 4D CT side concordance dataframes with a Modality column
combined_side_concordance_df = pd.concat(
    [side_concordance_spect_df.assign(Modality='SPECT/CT'),
     side_concordance_4dct_df.assign(Modality='4D CT Scan')],
    ignore_index=True
)[['Modality', 'Metric', 'Value']]

# Display combined results
display(combined_side_concordance_df)

,Modality,Metric,Value
0,SPECT/CT,Total SPECT/CT scans with localization,177
1,SPECT/CT,Scans matching surgical side or exact gland,163 (92.09%)
2,4D CT Scan,Total 4D CT scans with localization,171
3,4D CT Scan,Scans matching surgical side or exact gland,146 (85.38%)


---

### Interpretation

- **SPECT/CT:** Demonstrated a concordance rate of **92%**, indicating strong reliability in localizing the abnormal gland to the correct side or exact gland location.

- **4D CT:** Showed a concordance rate of **85%**, supporting its role as a highly useful imaging modality, though slightly lower than SPECT/CT in this cohort.

- **Clinical implication:** Both modalities provide valuable pre-operative localization data. The high concordance of SPECT/CT may reflect institutional expertise or protocol differences.

- **Next steps:** Compare these findings to PPV and sensitivity results to assess overall diagnostic performance and guide modality selection recommendations.